<a href="https://colab.research.google.com/github/ShiranZH/abcdefg/blob/master/code/feedback_prize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
os.chdir('/content/gdrive/MyDrive/2021_Fall_ML_Project')
!ls .

 data   feedback_prize.ipynb  'ML project.gdoc'   model   submission.csv


In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import random
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from sklearn.model_selection import *

In [4]:
# TRAIN_CSV = "../input/feedback-prize-2021/train.csv"
# SUB_CSV = "../input/feedback-prize-2021/sample_submission.csv"
# TRAIN_PATH = "../input/feedback-prize-2021/train"
# TEST_PATH = "../input/feedback-prize-2021/test"
TRAIN_CSV = "./data/train.csv"
SUB_CSV = "./data/sample_submission.csv"
TRAIN_PATH = "./data/train"
TEST_PATH = "./data/test"

df = pd.read_csv(TRAIN_CSV, dtype={'discourse_id':int, 'discourse_start':int, 'discourse_end':int})
df.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1622627660524,8,229,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1622627653021,230,312,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1622627671020,313,401,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1622627696365,402,758,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1622627759780,759,886,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [5]:
test_df = pd.read_csv(SUB_CSV)
test_df

,id,class,predictionstring
0,18409261F5C2,NaN,NaN
1,D46BCB48440A,NaN,NaN
2,0FB0700DAF44,NaN,NaN
3,D72CB1C11673,NaN,NaN
4,DF920E0A7337,NaN,NaN


In [6]:
test_names = []
test_texts = []
for test_file in tqdm(list(os.listdir(TEST_PATH))):
  test_names.append(test_file.replace('.txt', ''))
  test_texts.append(open(TEST_PATH+'/'+test_file, 'r').read())
test_texts = pd.DataFrame({'id':test_names, 'text':test_texts})
test_texts['text'] = test_texts['text'].apply(lambda x: x.split())
test_texts

100%|██████████| 5/5 [00:00<00:00,  5.45it/s]


,id,text
0,D46BCB48440A,"[When, people, ask, for, advice,they, sometime..."
1,18409261F5C2,"[80%, of, Americans, believe, seeking, multipl..."
2,D72CB1C11673,"[Making, choices, in, life, can, be, very, dif..."
3,0FB0700DAF44,"[During, a, group, project,, have, you, ever, ..."
4,DF920E0A7337,"[Have, you, ever, asked, more, than, one, pers..."


In [7]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [8]:
!pip install transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification
tokenizer = AutoTokenizer.from_pretrained("roberta-base", add_prefix_space=True)

     |████████████████████████████████| 3.4 MB 5.5 MB/s 
     |████████████████████████████████| 61 kB 229 kB/s 
     |████████████████████████████████| 895 kB 10.7 MB/s 
     |████████████████████████████████| 3.3 MB 35.7 MB/s 
     |████████████████████████████████| 596 kB 34.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [9]:
class MyDataset(Dataset):
  def __init__(self, df):
    self.df = df
  def __len__(self):
    return len(self.df)
  def __getitem__(self,idx):
    text = self.df.text.values[idx]
    return text

In [10]:
def collate_fn(data):
  tokenized_inputs = tokenizer(
      data,
      max_length = 512,
      padding = 'max_length',
      truncation = True,
      is_split_into_words = True,
      return_tensors = 'pt'
  )
  words = []
  for i in range(len(data)):
    word_ids = tokenized_inputs.word_ids(batch_index=i)
    words.append(word_ids)
  tokenized_inputs['word_ids'] = words
  return tokenized_inputs

In [11]:
test_loader = DataLoader(MyDataset(test_texts), batch_size=32, collate_fn=collate_fn, shuffle=False, num_workers=4)
batch = next(iter(test_loader))
batch

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


{'input_ids': tensor([[    0,   520,    82,  ...,     1,     1,     1],
        [    0,  1812,   207,  ...,    33,    57,     2],
        [    0, 11102,  5717,  ...,     1,     1,     1],
        [    0,  1590,    10,  ...,  1843,  1901,     2],
        [    0,  6319,    47,  ...,     6,    24,     2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'word_ids': [[None, 0, 1, 2, 3, 4, 4, 4, 5, 6, 7, 8, 9, 10, 11, 11, 12, 13, 14, 15, 15, 15, 15, 16, 17, 18, 19, 19, 20, 21, 22, 23, 24, 25, 26, 27, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 38, 39, 40, 41, 42, 42, 43, 44, 45, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 65, 66, 67, 68, 69, 70, 71, 72, 73, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 101, 102, 103, 104, 105, 1

In [12]:
model = AutoModelForTokenClassification.from_pretrained("roberta-base", num_labels=15)
model.load_state_dict(torch.load('./model/roberta-base_fold_0.pt'), False)
model.eval()

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able

RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm

In [13]:
y_pred = []
words = []
with torch.no_grad():
  tk = tqdm(test_loader, total=len(test_loader), position=0, leave=True)
  for step, batch in enumerate(tk):
    word_ids = batch['word_ids']
    words.extend(word_ids)
    batch = {k:v for k,v in batch.items() if k!='word_ids'}
    output = model(**batch).logits
    y_pred.extend(output.argmax(-1).cpu().numpy())
y_pred = np.array(y_pred)

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 1/1 [00:10<00:00, 10.05s/it]


In [14]:
labels = ['O', 'B-Lead', 'I-Lead', 'B-Postion', 'I-Position', 'B-Claim', 'I-Claim', 'B-Counterclaim', 'I-Counterclaim', 'B-Rebuttal', 'I-Rebuttal', 'B-Evidence', 'I-Evidence', 'B-Concluuding Statement', 'I-Concluding Statement']

In [15]:
print(y_pred)
print(len(labels))

[[14  1  2 ... 12 12 12]
 [12  3  3 ... 12 12 12]
 [14  1  2 ...  6  6  6]
 [12  1  2 ... 12 12  2]
 [12  1  2 ...  0  0  2]]
15


In [16]:
final_preds = []
for i in tqdm(range(len(test_texts))):
  idx = test_texts.id.values[i]
  pred = ['']*len(test_texts.text.values[i])
  for j in range(len(y_pred[i])):
    if words[i][j] != None:
      pred[words[i][j]] = labels[y_pred[i][j]%len(labels)]
  preds = []
  j = 0
  while j < len(pred):
    cls = pred[j]
    if cls=='O' or cls=='' or cls[0]=='I':
      j += 1
    else:
      end = j+1
      while end<len(pred) and pred[end].replace('B-','').replace('I-','')==cls.replace('B-','').replace('I-',''):
        end += 1
      if end-j > 5:
        final_preds.append((idx, cls.replace('B-','').replace('I-',''), ' '.join(map(str, list(range(j,end))))))
      j = end
final_preds[0]

100%|██████████| 5/5 [00:00<00:00, 707.90it/s]


('D46BCB48440A', 'Lead', '0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19')

In [17]:
sub = pd.DataFrame(final_preds)
sub.columns = test_df.columns
sub

,id,class,predictionstring
0,D46BCB48440A,Lead,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
1,D46BCB48440A,Claim,36 37 38 39 40 41 42
2,D46BCB48440A,Evidence,56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 7...
3,D46BCB48440A,Evidence,150 151 152 153 154 155 156 157 158 159 160 16...
4,D46BCB48440A,Evidence,223 224 225 226 227 228 229 230 231 232 233 23...
5,18409261F5C2,Evidence,55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70
6,18409261F5C2,Claim,139 140 141 142 143 144 145 146 147 148 149 15...
7,18409261F5C2,Claim,155 156 157 158 159 160 161
8,18409261F5C2,Evidence,162 163 164 165 166 167 168 169 170 171 172 17...
9,D72CB1C11673,Lead,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...


In [18]:
sub.to_csv("submission.csv", index=False)